# Interactive simulation with Duty Cycle modification

This python script example helps to understand how to change the value of a parameter during the simulation interactively (pseudo real time).

In this case, the **duty cycle** of a Buck converter will be modified suddenly at any time and the impact of this change will be observed on the output voltage through an oscilloscope. 

The library called **ipywidgets** is used to create this interactive display in pseudo real-time.

In [5]:
# Load required module
from aesim.simba import DesignExamples
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

# Result tables initilization
table_t = []
table_Vout = []

# Load project
Buck = DesignExamples.BuckConverter()
R1 = Buck.Circuit.GetDeviceByName('R1')
for scope in R1.Scopes:
    scope.Enabled=True
    
# Get the job object and solve the system
Buck.TransientAnalysis.NumberOfPointsToSimulate = 1000 # emulate the time by using points
Buck.TransientAnalysis.StopAtSteadyState=False
job = Buck.TransientAnalysis.NewJob()

def run_simulation(duty_cycle, count):
    PWM = Buck.Circuit.GetDeviceByName('C1')
    PWM.DutyCycle=duty_cycle        # set duty cycle
    status = job.Run()              # simulate
    t = job.TimePoints              # get results
    Vout = job.GetSignalByName('R1 - Voltage').DataPoints

    if len(table_t) < 25000:
        # remove all data as t and Vout contain result from t = 0
        del table_t[0:len(t)]
        del table_Vout[0:len(t)]
        table_t.extend(t)
        table_Vout.extend(Vout)
    else:
        # Fill tables with the length of simulation (1000 points)
        table_t.extend(t)
        table_Vout.extend(Vout)
        del table_t[0:len(t)]
        del table_Vout[0:len(Vout)]

    
    # graph initialization
    fig, ax = plt.subplots()
    ax.set_title('Buck')
    ax.set_ylabel('Vout (V)')
    ax.set_xlabel('time (s)')
    ax.set_ylim((0, 70))
    ax.plot(table_t, table_Vout)

    plt.show()

w=widgets.interact(run_simulation,count=widgets.Play(min=0, max=10), duty_cycle=(0.0,1.0,0.1))
w    

interactive(children=(FloatSlider(value=0.5, description='duty_cycle', max=1.0), Play(value=0, description='co…

<function __main__.run_simulation(duty_cycle, count)>